# Задание по программированию: Размещение баннеров

Представим, что международное круизное агентство Carnival Cruise Line решило себя разрекламировать с помощью баннеров и обратилось для этого к вам. Чтобы протестировать, велика ли от таких баннеров польза, их будет размещено всего 20 штук по всему миру. Вам надо выбрать 20 таких локаций для размещения, чтобы польза была большой и агентство продолжило с вами сотрудничать.

Агентство крупное, и у него есть несколько офисов по всему миру. Вблизи этих офисов оно и хочет разместить баннеры — легче договариваться и проверять результат. Также эти места должны быть популярны среди туристов.

Для поиска оптимальных мест воспользуемся базой данных крупнейшей социальной сети, основанной на локациях — Foursquare.

Часть открытых данных есть, например, на сайте archive.org:

https://archive.org/details/201309_foursquare_dataset_umn

Скачаем любым удобным образом архив fsq.zip с этой страницы.



In [79]:
import os, urllib
from io import BytesIO
from zipfile import ZipFile
from pathlib import Path

path_to_zip = '../datafiles/fsq.zip'

if not (Path(path_to_zip)).exists():
    urllib.request.urlretrieve("https://archive.org/download/201309_foursquare_dataset_umn/fsq.zip", path_to_zip)

with ZipFile(path_to_zip, "r") as zip_ref:
    zip_ref.extractall("../datafiles/fsq/")
    
path_to_checkins = '../datafiles/fsq/umn_foursquare_datasets/checkins.dat'

Нас будет интересовать файл checkins.dat. Открыв его, увидим следующую структуру:

In [82]:
dat_file = open(path_to_checkins, 'r')
for i in range(5):
    print(dat_file.readline())
dat_file.close()

   id    | user_id | venue_id |     latitude      |     longitude     |     created_at      

---------+---------+----------+-------------------+-------------------+---------------------

  984301 | 2041916 |     5222 |                   |                   | 2012-04-21 17:39:01

  984222 |   15824 |     5222 |        38.8951118 |       -77.0363658 | 2012-04-21 17:43:47

  984315 | 1764391 |     5222 |                   |                   | 2012-04-21 17:37:18



Для удобной работы с этим документом преобразуем его к формату csv, удалив строки, не содержащие координат — они неинформативны для нас:



In [83]:
path_to_csv = '../datafiles/checkins.csv'

if not (Path(path_to_csv)).exists():
    csv_file = open(csv_file_path, "w")
    
    dat_file = open(path_to_checkins, 'r')
    dat_file.readline() # пропускаем шапку
    dat_file.readline()
    for line in dat_file:
        line = line.split("|")
        line = [w.strip() for w in line]
        if (len(line) > 4 and line[3] != '' and line[4] != ''):        
            csv_file.write(line[3]+ "|" + line[4] + "\n")

    csv_file.close()

С помощью pandas построим DataFrame и убедимся, что все 396634 строки с координатами считаны успешно.



In [84]:
import pandas as pd
import numpy as np

#df = pd.read_csv(path_to_csv, 
#                 sep='|',
#                 names=['id','user_id','venue_id','latitude','longitude','created_at'],
#                 dtype={'id': np.int32, 'user_id': np.int32, 'venue_id': np.int32,
#                       'latitude': np.float32,'longitude' : np.float32}
#                )
#df.created_at = df.created_at.apply(pd.to_datetime)

df = pd.read_csv(path_to_csv, sep='|', names=['latitude','longitude'], dtype={'latitude': np.float32,'longitude' : np.float32})
print(df.columns)
print(df.dtypes)

Index(['latitude', 'longitude'], dtype='object')
latitude     float32
longitude    float32
dtype: object


In [88]:
print(df.head(100000))

        latitude   longitude
0      38.895111  -77.036369
1      33.800743  -84.410522
2      45.523453 -122.676208
3      40.764462 -111.904564
4      33.448376 -112.074036
...          ...         ...
99995  33.575001 -117.725555
99996  37.629349 -122.400085
99997  29.762884  -95.383064
99998  32.802956  -96.769920
99999  37.774929 -122.419418

[100000 rows x 2 columns]


_________________________________

Теперь необходимо **кластеризовать данные координаты, чтобы выявить центры скоплений туристов**. Поскольку баннеры имеют сравнительно небольшую площадь действия, нам нужен алгоритм, позволяющий ограничить размер кластера и не зависящий от количества кластеров.

Эта задача — хороший повод познакомиться с алгоритмом MeanShift, который мы обошли стороной в основной части лекций. Его описание при желании можно посмотреть в sklearn user guide, а чуть позже появится дополнительное видео с обзором этого и некоторых других алгоритмов кластеризации. Используйте MeanShift, указав bandwidth=0.1, что в переводе из градусов в метры колеблется примерно от 5 до 10 км в средних широтах.



**Примечание**:на 396634 строках кластеризация будет работать долго. Быть очень терпеливым не возбраняется — результат от этого только улучшится. Но для того, чтобы сдать задание, понадобится сабсет из первых 100 тысяч строк. Это компромисс между качеством и затраченным временем. Обучение алгоритма на всём датасете занимает около часа, а на 100 тыс. строк — примерно 2 минуты, однако этого достаточно для получения корректных результатов.

Некоторые из получившихся кластеров содержат слишком мало точек — такие кластеры не интересны рекламодателям. Поэтому надо определить, какие из кластеров содержат, скажем, больше 15 элементов. Центры этих кластеров и являются оптимальными для размещения.



In [91]:
%%time
from sklearn.cluster import MeanShift, estimate_bandwidth
from sklearn.datasets import make_blobs

clustering = MeanShift(bandwidth=0.1).fit(df.head(100000))

CPU times: user 2min 19s, sys: 0 ns, total: 2min 19s
Wall time: 2min 19s


In [96]:
labels = clustering.labels_
cluster_centers = clustering.cluster_centers_

labels_unique = np.unique(labels)
n_clusters_ = len(labels_unique)

print(clustering.cluster_centers_)

[[  40.719643  -73.994316]
 [  33.452442 -112.00194 ]
 [  33.385387 -111.86146 ]
 ...
 [ -37.822983  145.18118 ]
 [ -41.292496  174.77324 ]
 [ -45.031162  168.66264 ]]


При желании увидеть получившиеся результаты на карте можно передать центры получившихся кластеров в один из инструментов визуализации. Например, сайт mapcustomizer.com имеет функцию Bulk Entry, куда можно вставить центры полученных кластеров в формате:
<pre>
38.8951118,-77.0363658

33.800745,-84.41052

45.5234515,-122.6762071
</pre>


Как мы помним, 20 баннеров надо разместить близ офисов компании. Найдем на Google Maps по запросу Carnival Cruise Line адреса всех офисов:

<pre>
33.751277, -118.188740 (Los Angeles)

25.867736, -80.324116 (Miami)

51.503016, -0.075479 (London)

52.378894, 4.885084 (Amsterdam)

39.366487, 117.036146 (Beijing)

-33.868457, 151.205134 (Sydney)
</pre>

_______________________

Осталось определить 20 ближайших к ним центров кластеров. Т.е. посчитать дистанцию до ближайшего офиса для каждой точки и выбрать 20 с наименьшим значением.

**Примечание**: при подсчете расстояний и в кластеризации можно пренебречь тем, что Земля круглая, так как в точках, расположенных близко друг к другу погрешность мала, а в остальных точках значение достаточно велико.

Для сдачи задания выберите из получившихся 20 центров тот, который наименее удален от ближайшего к нему офиса. Ответ в этом задании — широта и долгота этого центра, записанные через пробел.